In [71]:
""" given
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')
"""

" given\nimport matplotlib.pyplot as plt\nimport mplleaflet\nimport pandas as pd\n\ndef leaflet_plot_stations(binsize, hashid):\n\n    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))\n\n    station_locations_by_hash = df[df['hash'] == hashid]\n\n    lons = station_locations_by_hash['LONGITUDE'].tolist()\n    lats = station_locations_by_hash['LATITUDE'].tolist()\n\n    plt.figure(figsize=(8,8))\n\n    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)\n\n    return mplleaflet.display()\n\nleaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')\n"

In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [73]:
df = pd.read_csv(r'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
df.head()

,ID,Date,Element,Data_Value
0,USW00094889,2014-11-12,TMAX,22
1,USC00208972,2009-04-29,TMIN,56
2,USC00200032,2008-05-26,TMAX,278
3,USC00205563,2005-11-11,TMAX,139
4,USC00200230,2014-02-27,TMAX,-106


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165085 entries, 0 to 165084
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   ID          165085 non-null  object
 1   Date        165085 non-null  object
 2   Element     165085 non-null  object
 3   Data_Value  165085 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 5.0+ MB


In [75]:
df.isnull().sum() # check for missing values

ID            0
Date          0
Element       0
Data_Value    0
dtype: int64

In [76]:
# parse Date into 'Year' & 'Date2'
df['Year'] = df['Date'].apply(lambda x: x[:4])
df['Date2'] = df['Date'].apply(lambda x: x[-5:])
df.sort_values(by=['ID', 'Date'], inplace=True)

In [77]:
# drop leap day (02-29)
df = df[df['Date2'] != '02-29']


In [78]:
# check if leap day dropped?
print('02-29' in df.Date2)

False


In [79]:
# separate 2005-2014 bin from 2015
df_15 = df[df['Year'] == '2015']
df_0514 = df[df['Year'] != '2015']

In [80]:
# TMIN & TMAX for dates over 2005-2014


,ID,Date,Element,Data_Value,Year,Date2
84041,USW00094889,2015-12-29,TMAX,50,2015,12-29
50750,USW00094889,2015-12-30,TMAX,6,2015,12-30
50751,USW00094889,2015-12-30,TMIN,-5,2015,12-30
61120,USW00094889,2015-12-31,TMAX,-5,2015,12-31
61135,USW00094889,2015-12-31,TMIN,-21,2015,12-31


,ID,Date,Element,Data_Value,Year,Date2
56613,USW00094889,2014-12-29,TMAX,6,2014,12-29
60944,USW00094889,2014-12-30,TMIN,-105,2014,12-30
60945,USW00094889,2014-12-30,TMAX,-38,2014,12-30
45681,USW00094889,2014-12-31,TMIN,-138,2014,12-31
45683,USW00094889,2014-12-31,TMAX,-49,2014,12-31
